In [149]:
import pandas as pd
from pocketbase import PocketBase
import string
from slugify import slugify
from openai import OpenAI

In [164]:
pb = PocketBase('http://127.0.0.1:8080')
pb.admins.auth_with_password("cogitoteamdev@gmail.com", "fFXYNpQjv87V")

openai = OpenAI(api_key='sk-')

In [61]:
t = pd.read_csv('themes.csv', sep=';')
p1 = pd.read_csv('part1.csv', sep=';')
p2 = pd.read_csv('part2.csv', sep=';')
p3 = pd.read_csv('part3.csv', sep=';')

for tt in [t, p1, p2, p3]:
    tt.loc[tt.Theme == 'Home', 'Theme'] = 'Home & Hometown'
    tt.loc[tt.Theme == 'places', 'Theme'] = 'Places'
    tt.loc[tt.Theme == 'Clothes & accessories', 'Theme'] = 'Clothes & Accessories'
    tt.loc[tt.Theme == 'Family, friends & Other people', 'Theme'] = 'Family, Friends & Other People'
    tt.loc[tt.Theme == 'Scools & Studies', 'Theme'] = 'Schools & Studies'
    tt.loc[tt.Theme == 'Free time, hobbies & Skills', 'Theme'] = 'Free time, Hobbies & Skills'

    if tt is t:
        continue

    tt.loc[tt["Global theme"] == 'Home', 'Global theme'] = 'Home & Hometown'
    tt.loc[tt["Global theme"] == 'places', 'Global theme'] = 'Places'
    tt.loc[tt["Global theme"] == 'Clothes & accessories', 'Global theme'] = 'Clothes & Accessories'
    tt.loc[tt["Global theme"] == 'Family, friends & Other people', 'Global theme'] = 'Family, Friends & Other People'
    tt.loc[tt["Global theme"] == 'Scools & Studies', 'Global theme'] = 'Schools & Studies'
    tt.loc[tt["Global theme"] == 'Free time, hobbies & Skills', 'Global theme'] = 'Free time, Hobbies & Skills'

In [62]:
t

,Theme
0,Home & Hometown
1,Places
2,"Nature, Animals & The Environment"
3,Travel & Transport
4,"Family, Friends & Other People"
5,Schools & Studies
6,"Free time, Hobbies & Skills"
7,Technology
8,Media & Entertainment
9,Body & Mind


In [66]:
for i, row in t.iterrows():
    title = row.Theme.strip()
    slug = slugify(title)

    try:
        with open('themes/' + title + '.svg', 'r') as f:
            icon = f.read().strip()
    except:
        print(title)
        icon = ''

    pb.collection('topics').create({
        'title': title,
        'slug': slug,
        'essentials': '{}',
        'icon': icon,
    })

Places
Nature, Animals & The Environment
Family, Friends & Other People
Free time, Hobbies & Skills
Work, Business, Money & Shopping


In [67]:
tasks = pd.concat([p1, p2, p3])

In [118]:
tasks

,Part,Global theme,Theme,Question,link
0,1,Home & Hometown,Accommodation,Do you live in a house or an apartment/flat?\n...,https://yourielts.ru/prepare-for-ielts/ielts-s...
1,1,Home & Hometown,Home decoration,What kind of decoration do you prefer?\nHow is...,https://yourielts.ru/prepare-for-ielts/ielts-s...
2,1,Home & Hometown,Hometown,Tell me about your hometown.\nWhat’s your home...,https://yourielts.ru/prepare-for-ielts/ielts-s...
3,1,Home & Hometown,Your Country,Where is your country located?\nWhich part of ...,https://yourielts.ru/prepare-for-ielts/ielts-s...
4,1,Places,Crowded Places,How do you feel when you go to crowded places?...,https://markteacher.com/latest-speaking-questi...
...,...,...,...,...,...
59,3,"Work, Business, Money & Shopping",Business,"In your opinion, do business people have to wo...",https://www.bestmytest.com/ielts/speaking
60,3,"Work, Business, Money & Shopping",Company,What is the difference between big companies a...,https://www.bestmytest.com/ielts/speaking
61,3,"Work, Business, Money & Shopping",Money,Is money important to you?\nWhat is the relati...,https://www.bestmytest.com/ielts/speaking
62,3,"Work, Business, Money & Shopping",Shopping,Is shopping a popular activity in your country...,https://www.bestmytest.com/ielts/speaking


In [ ]:
part_2_template = """
<h5>{title}</h5>
<br/>
<p>You should say:</p>
<ul>
    <li>{question_t}</li>
</ul>
<br/>
<p>And {last_question}</p>
"""

for i, row in tasks.iterrows():
    topic = row["Global theme"].strip()
    
    key_topic = t.loc[t.Theme == topic, 'Theme'].values[0]
    f = f'title="${key_topic}"'
    db_topic = pb.collection("topics").get_first_list_item(f'title="{key_topic}"')
    topic_id = db_topic.id

    slug = slugify(row.Theme.strip())

    if row.Part == 2:
        res = openai.chat.completions.create(messages=[
            {
                'role': 'system',
                'content': f"""
You need to improve exam questions.Fix grammar and lexical mistakes and return string in this format:
{part_2_template}

Do not mark it like "html", just raw html component.

This is the raw question:
{row.Question}
"""            
            }
        ], model='gpt-4o-mini')

        questions_content = res.choices[0].message.content
            
    else:
        res = openai.chat.completions.create(messages=[
            {
                'role': 'system',
                'content': f"""
You need to improve exam questions. Fix grammar and lexical mistakes and return html string with one <ul> tag and questions inside in <li> tags. 
Do not mark it like "html", just raw html component.

This is the raw question:
{row.Question}
"""            
            }
        ], model='gpt-4o-mini')

        questions_content = res.choices[0].message.content


    pb.collection('tasks').create({
        'questions': questions_content,
        'examType': 'general',
        'topicID': topic_id,
        'section': 'speaking',
        'part': row.Part,
        'title': row.Theme,
        'slug': slug,
    })

<h5>Describe a Problem in Your City</h5>
<br/>
<p>You should say:</p>
<ul>
    <li>What the problem is</li>
    <li>How serious it is</li>
    <li>What causes this problem</li>
</ul>
<br/>
<p>And discuss what can be done to solve or reduce the effects of this problem.</p>
<h5>Describe a Shopping Street in Your Hometown</h5>
<br/>
<p>You should say:</p>
<ul>
    <li>What items you can purchase there</li>
    <li>What the most popular shops are</li>
    <li>Any changes that have occurred over the years</li>
</ul>
<br/>
<p>And discuss whether you think shopping there will become more or less popular in the future, and why.</p>
<h5>Describe a Shop in Your Hometown</h5>
<br/>
<p>You should say:</p>
<ul>
    <li>Where it is located</li>
    <li>What it sells</li>
    <li>What the advantages of shopping there are</li>
</ul>
<br/>
<p>And whether you think it will be popular in the future, and why.</p>
<h5>Transportation System in Your Hometown</h5>
<br/>
<p>You should say:</p>
<ul>
    <li>Wha